In [6]:
import pandas as pd
df=pd.read_csv("data/gemstone.csv")

In [7]:
X=df.drop(labels=['id'],axis=1)

In [8]:
Y=df[['price']]

In [32]:
categorical_cols=X.select_dtypes(include='object').columns
numerical_cols=X.select_dtypes(exclude='object').columns

In [33]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [34]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer



In [35]:
##Numerical pipeline

num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())

    ]
)

cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)


preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [36]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test =train_test_split(X,Y,test_size=0.30,random_state=42)

In [37]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [38]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [40]:
regression=LinearRegression()
regression.fit(x_train,y_train)

LinearRegression()

In [43]:
regression.coef_

array([[ 1.16430235e-11,  3.48751609e-13,  5.47883118e-13,
        -5.13776518e-12, -5.23582565e-13, -2.08122554e-12,
         4.04055821e+03,  4.93345211e-13, -1.05434467e-12,
         2.04290113e-13]])

In [44]:
regression.intercept_

array([3976.8787389])

In [45]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [47]:
## Train multiple models

models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    #Make Predictions
    y_pred=model.predict(x_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 5.6347330295078705e-12
MAE: 4.236761069786453e-12
R2 score 100.0


Lasso
Model Training Performance
RMSE: 1.2992649662529592
MAE: 0.8255484912018893
R2 score 99.99998955319947


Ridge
Model Training Performance
RMSE: 0.1186099864923161
MAE: 0.07891958788274715
R2 score 99.99999991293765


Elasticnet
Model Training Performance
RMSE: 1031.8687070964268
MAE: 713.9023832648608
R2 score 93.41073869556055


